<a href="https://colab.research.google.com/github/afonsofarias/Arduino---Semaforo/blob/main/Processamento_de_linguagem_natural_utilizando_pouco_c%C3%B3digo_e_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalando bibliotecas

In [ ]:
!pip install -q transformers==4.35.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 27.1 MB/s eta 0:00:00


`transformers`: A biblioteca transformers, desenvolvida pela Hugging Face, fornece modelos de PLN de ponta, como BERT e GPT-3. Esses modelos são pré-treinados em grandes volumes de texto e podem ser ajustados para tarefas específicas de PLN.

https://huggingface.co/

# Importando as bibliotecas

In [ ]:
from transformers import pipeline
import torch

O módulo `pipeline` da biblioteca transformers simplifica o processo de uso de modelos pré-treinados para tarefas específicas de PLN. Ele fornece uma interface simples e conveniente para realizar inferências de texto usando modelos como os disponíveis na Hugging Face, abstraindo grande parte da complexidade do código subjacente.

A biblioteca `torch` será utilizada no código apenas para verificar se a GPU está disponível no ambiente para processamento mais rápido.

# Carregando o modelo

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
classifier = pipeline("zero-shot-classification",
                      model="MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7",
                      device=device)

A função pipeline é usada para criar um classificador de zero-shot passando o nome da tarefa `“zero-shot-classification”`. No exemplo, estamos usando o modelo `“MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"`. O dispositivo (CPU ou GPU) é especificado com base no valor da variável device.

Multilingual - https://huggingface.co/MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7

Inglês - https://huggingface.co/facebook/bart-large-mnli

Agora o modelo está pronto para fazer inferências! Você precisará do texto que deseja classificar e dos rótulos que deseja que o modelo atribua à sentença (rótulos candidatos). O modelo fornecerá as probabilidades da sequência pertencer a cada rótulo candidato.

# Realizando a classificação

In [ ]:
candidate_labels = ["Política", "Negócios", "Esportes", "Entretenimento", "Outros", "acidente", "problema", "engenharia", "arquitetura",
                    "cultura", "atividades", "cotidiano", "negativo", "positivo"]
sequences = ['''Casas desabam e muro de condomínio cai após chuvas em São Luís''']

results = classifier(sequences, candidate_labels)
print(results)

# [
#    {
#       "sequence":"'Vergonhosa página da diplomacia de Israel', diz Mauro Vieira sobre postura israelense após fala de Lula",
#       "labels":[
#          "Política",
#          "Negócios",
#          "Outros",
#          "Entretenimento",
#          "Esportes"
#       ],
#       "scores":[
#          0.529527485370636,
#          0.24372632801532745,
#          0.13887633383274078,
#          0.0627482607960701,
#          0.02512160874903202
#       ]
#    }
# ]

[{'sequence': 'Casas desabam e muro de condomínio cai após chuvas em São Luís', 'labels': ['negativo', 'atividades', 'arquitetura', 'engenharia', 'problema', 'acidente', 'Outros', 'Negócios', 'cultura', 'Política', 'Entretenimento', 'cotidiano', 'Esportes', 'positivo'], 'scores': [0.5184404850006104, 0.15481950342655182, 0.06538498401641846, 0.06215418130159378, 0.04195767268538475, 0.0286422036588192, 0.02455381117761135, 0.024047788232564926, 0.022603077813982964, 0.019015464931726456, 0.015079226344823837, 0.014562963508069515, 0.006390289403498173, 0.0023482623510062695]}]


In [ ]:
print("Entrada:", results[0]["sequence"])
print("Rótulo:", results[0]["labels"][0])
print("Confiança:", results[0]["scores"][0])

Entrada: Casas desabam e muro de condomínio cai após chuvas em São Luís
Rótulo: negativo
Confiança: 0.5184404850006104


O texto está muito relacionado ao rótulo Política, de acordo com a previsão do modelo, com um escore de confiança de 0,53. O que está correto!

Esses escores são probabilidades e somam 1,0, indicando a probabilidade estimada pelo modelo para cada rótulo.

Neste resultado, o modelo está mais confiante em sua previsão para Política e menos certo sobre as outras categorias. Isso significa que às vezes o modelo pode ficar confuso se as classes forem similares, mas neste caso, ele acertou.

# Encapsulando em uma função

Pronto, agora a lógica de classificação estará dentro de uma função. Onde a mesma pode ser implementada em APIs em Python (ou em outras linguagens) ou em algum provedor de computação em nuvem para ser consumido como microsserviço!

In [ ]:
def predict_text_label(text, candidate_labels):
  """
  Classifica um texto usando o modelo de classificação zero-shot.

  Args:
    text: O texto a ser classificado.
    candidate_labels: Uma lista de rótulos candidatos.

  Returns:
    Um dicionário contendo o texto, o rótulo previsto e a confiança da previsão.
  """

  sequences = [text]
  results = classifier(sequences, candidate_labels)
  predicted_label = results[0]["labels"][0]
  confidence = results[0]["scores"][0]
  return {
      "text": text,
      "predicted_label": predicted_label,
      "confidence": confidence
  }


In [ ]:
predict_text_label(sequences[0], candidate_labels)

{'text': 'Casas desabam e muro de condomínio cai após chuvas em São Luís',
 'predicted_label': 'desastres',
 'confidence': 0.9582889676094055}

Você pode ler mais sobre esse conteúdo em: https://medium.com/wearesinch/easy-text-classification-using-minimal-code-and-ai-d1a85ae0f321